In [ ]:
from tensorflow.keras.models import load_model

gru_model = load_model('gru_model.h5')


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
import joblib
import os

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle



gru_model = load_model('gru_model.h5')
print("Model loaded from 'gru_model.h5'")
label_encoder = joblib.load('gru_label_encoder.pkl')
model = load_model('gru_model.h5')

def process_video(file_path, class_name,df,timesteps=30):
    cap = cv2.VideoCapture(file_path)
    pose = mp.solutions.pose.Pose()
    frame_data = []
    row = []
    frame_count=0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            try:
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x, 
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x, 
                             landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                              landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].y]
                right_hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]

                right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                
                left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)
                
                angles = [right_shoulder_angle, right_elbow_angle, right_hip_angle, right_knee_angle,
                          left_shoulder_angle, left_elbow_angle, left_hip_angle, left_knee_angle]

                frame_data.append(angles)

                # If we have enough frames, make a prediction
                if len(frame_data) == timesteps:
                    input_data = np.array(frame_data).reshape(1, timesteps, -1)
                    prediction = model.predict(input_data)
                    predicted_class = np.argmax(prediction, axis=-1)
                    predicted_label = label_encoder.inverse_transform(predicted_class)[0]

                    # Display the predicted class on the frame
                    cv2.putText(frame, f'Predicted Class: {predicted_label}', (50, 50), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

                    names = label_encoder.classes_
                    # Remove the oldest frame
                    frame_data.pop(0)
                    row.append({
                        'frame': frame_count,
                        '1_prob': prediction[0][0],
                        '2_prob': prediction[0][1],
                        '3_prob': prediction[0][2],
                        '4_prob': prediction[0][3],
                        '5_prob': prediction[0][4],
                        'ans':predicted_label,
                        'class_name': class_name
                    })
                    
            except IndexError:
                pass
    cap.release()
    df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)
    return df
    
if __name__ == "__main__":
    video_folder = 'vid'  
    output_folder = 'csv'  

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

 
    all_data = pd.DataFrame()


    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0]  
            file_path = os.path.join(video_folder, file_name)

            
            all_data = process_video(file_path, class_name, all_data)

    
    output_file = os.path.join(output_folder, 'gru_prob.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


In [7]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
import joblib
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle




label_encoder = joblib.load('label_encoder.pkl')
model = load_model('lstm_model2.h5')
data = pd.read_csv('csv/output.csv')
data['class_name'] = label_encoder.fit_transform(data['class_name'])

scaler = StandardScaler()
scaler.fit(data[['right_shoulder_angle',
            'right_elbow_angle',
            'right_hip_angle',
            'right_knee_angle',
            'left_shoulder_angle',
            'left_elbow_angle',
            'left_hip_angle',
            'left_knee_angle']])

def process_video(file_path, class_name,df,sequence_length=40):
    cap = cv2.VideoCapture(file_path)
    pose = mp.solutions.pose.Pose()
    frame_data = []
    row = []
    frame_count=0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            try:
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x, 
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x, 
                             landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                              landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].y]
                right_hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]

                right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                
                left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)
                
                angles = [right_shoulder_angle, right_elbow_angle, right_hip_angle, right_knee_angle,
                          left_shoulder_angle, left_elbow_angle, left_hip_angle, left_knee_angle]

                frame_data.append(angles)

                if len(frame_data) >= sequence_length:
                    sequence = scaler.transform(frame_data[-sequence_length:])
                    sequence = np.expand_dims(sequence, axis=0)
                    prediction = model.predict(sequence)
                    predicted_class = np.argmax(prediction, axis=1)
                    predicted_label = label_encoder.inverse_transform(predicted_class)[0]

                    
                    cv2.putText(frame, f'Predicted Class: {predicted_label}', (50, 50), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

                    names = label_encoder.classes_
                    
                    frame_data.pop(0)
                    row.append({
                        'frame': frame_count,
                        '1_prob': prediction[0][0],
                        '2_prob': prediction[0][1],
                        '3_prob': prediction[0][2],
                        '4_prob': prediction[0][3],
                        '5_prob': prediction[0][4],
                        'ans':predicted_label,
                        'class_name': class_name
                    })
                    
            except IndexError:
                pass
    cap.release()
    df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)
    return df
    
if __name__ == "__main__":
    video_folder = 'test'  
    output_folder = 'csv'  

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    
    all_data = pd.DataFrame()

    
    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0] 
            file_path = os.path.join(video_folder, file_name)

           
            all_data = process_video(file_path, class_name, all_data)

   
    output_file = os.path.join(output_folder, '2lstm_prob.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 394ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step

C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

Saved all angles to csv\2lstm_prob.csv


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
import joblib
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder

def calculate_angle_2d(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    
    if angle > 180.0:
        angle = 360.0 - angle
        
    return angle



test_label_encoder = joblib.load('test_label_encoder.pkl')
mlp_model = joblib.load('mlp_model.pkl')
def process_video(file_path, class_name,df,sequence_length=40):
    cap = cv2.VideoCapture(file_path)
    pose = mp.solutions.pose.Pose()
    frame_data = []
    row = []
    frame_count=0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(image)

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark

            try:
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x, 
                                 landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                left_wrist = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y]
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x, 
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x, 
                             landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].x, 
                              landmarks[mp.solutions.pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                  landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                right_wrist = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y]
                right_knee = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].x,
                              landmarks[mp.solutions.pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ANKLE.value].y]
                right_hip = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_HIP.value].y]

                right_shoulder_angle = calculate_angle_2d(right_hip, right_shoulder, right_elbow)
                right_elbow_angle = calculate_angle_2d(right_shoulder, right_elbow, right_wrist)
                right_hip_angle = calculate_angle_2d(right_knee, right_hip, right_shoulder)
                right_knee_angle = calculate_angle_2d(right_hip, right_knee, right_ankle)
                                
                left_shoulder_angle = calculate_angle_2d(left_hip, left_shoulder, left_elbow)
                left_elbow_angle = calculate_angle_2d(left_shoulder, left_elbow, left_wrist)
                left_hip_angle = calculate_angle_2d(left_knee, left_hip, left_shoulder)
                left_knee_angle = calculate_angle_2d(left_hip, left_knee, left_ankle)
                
                angles = [right_shoulder_angle, right_elbow_angle, right_hip_angle, right_knee_angle,
                          left_shoulder_angle, left_elbow_angle, left_hip_angle, left_knee_angle]
                
                data_test = np.array([right_shoulder_angle,right_elbow_angle,right_hip_angle,
                       right_knee_angle,left_shoulder_angle,left_elbow_angle,left_hip_angle,left_knee_angle])
                pose_data = data_test.flatten()
                pose_data = data_test.reshape(1, -1)
                
                mlp_prediction = mlp_model.predict(pose_data)[0]
                mlp_class_name = test_label_encoder.inverse_transform([mlp_prediction])[0]
                row.append({
                    'frame': frame_count,
                    'ans':mlp_class_name,
                    'class_name': class_name
                })
                    
            except IndexError:
                pass
    cap.release()
    df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)
    return df
    
if __name__ == "__main__":
    video_folder = 'vid'  
    output_folder = 'csv'  

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_data = pd.DataFrame()

    # Process each video
    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0] 
            file_path = os.path.join(video_folder, file_name)

            
            all_data = process_video(file_path, class_name, all_data)

    
    output_file = os.path.join(output_folder, 'mlp_prob.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')


In [5]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
import joblib
import os
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from mediapipe.tasks.python.audio import audio_classifier
from ultralytics import YOLO

class Result:
    def __init__(self, names):
        self.names = names
        
model = YOLO('yolomo.pt')

def process_video(file_path, class_name,df):
    frame_data = []
    row = []
    frame_count=0
    mp_drawing = mp.solutions.drawing_utils
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.3, min_tracking_confidence=0.3)
    cap = cv2.VideoCapture(file_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        exit()
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame_count += 1
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
        pose_results = pose.process(frame_rgb)
    
        if pose_results.pose_landmarks:
    
            mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    
            landmarks = pose_results.pose_landmarks.landmark
            
            results = model(frame)
            names = results[0].names
            top1 = results[0].probs.top1
            if top1 == 0:
                classname = names[0]    # bicep
            if top1 == 1:
                classname = names[1]    # dumbbell row 
            if top1 == 2:
                classname = names[2]    # incline dumbbell press
            if top1 == 3:
                classname = names[3]    # lat pull down
            if top1 == 4:
                classname = names[4]    # squat
                
            cv2.putText(frame, f'Class: {classname}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            row.append({
                'frame': frame_count,
                'ans':classname,
                'class_name': class_name
            })
    cap.release()
    df = pd.concat([df, pd.DataFrame(row)], ignore_index=True)
    return df
    
if __name__ == "__main__":
    video_folder = 'vid'  
    output_folder = 'csv'  
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    all_data = pd.DataFrame()


    for file_name in os.listdir(video_folder):
        if file_name.endswith('.mp4'):
            class_name = file_name.split('_')[0]  
            file_path = os.path.join(video_folder, file_name)

            all_data = process_video(file_path, class_name, all_data)

    # Save the consolidated DataFrame to a single CSV file
    output_file = os.path.join(output_folder, 'yolo_prob.csv')
    all_data.to_csv(output_file, index=False)
    print(f'Saved all angles to {output_file}')

C:\Users\ideapad GAMING\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 83.8ms
Speed: 43.3ms preprocess, 83.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 53.0ms
Speed: 38.0ms preprocess, 53.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 58.0ms
Speed: 34.0ms preprocess, 58.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 64.0ms
Speed: 25.0ms preprocess, 64.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 60.0ms
Speed: 32.0ms preprocess, 60.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 bicep 1.00, squat 0.00, lat 0.00, row 0.00, incline 0.00, 35.0ms
Speed: 29.0ms preprocess, 35.0ms inference, 0.0ms